# AI Content Automation Pipeline — Demo Version
This notebook is a sanitized public demo. Sensitive logic, prompts, credentials, and third‑party integrations have been removed or abstracted.

In [ ]:
import os
if not os.path.exists('/content/drive/MyDrive'):
    from google.colab import drive
    drive.mount('/content/drive')
else:
    print("✅ Drive already mounted")


In [ ]:
!pip install external_service moviepy gspread oauth2client requests

import pandas as pd
import requests
import json
import os
import external_service as mp
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from external_service import external_service
from google.colab import drive
from external_service import external_service


In [ ]:
import os
import re
import pandas as pd

# -------------------------
# PATHS
# -------------------------
drive_root = "/content/drive/MyDrive/external_service"
script_folder = os.path.join(drive_root, "script")
rtf_file = os.path.join(script_folder, "script_22.txt")
output_csv = os.path.join(script_folder, "script_chunks.csv")

# -------------------------
# LOAD FILE
# -------------------------
if not os.path.exists(rtf_file):
    raise FileNotFoundError("❌ script_22.txt not found in script folder")

with open(rtf_file, "r", encoding="utf-8", errors="ignore") as f:
    content = f.read()

# -------------------------
# CLEAN RTF (remove tags)
# -------------------------
clean_text = re.sub(r"{\\.*?}|\\[A-Za-z]+\d*|[{}]", "", content)
clean_text = re.sub(r"\s+", " ", clean_text).strip()

# -------------------------
# FIX: Replace semicolons so CSV does NOT break
# -------------------------
clean_text = clean_text.replace(";", ".")   # <--- important fix

# -------------------------
# SPLIT INTO SENTENCES
# -------------------------
sentences = re.split(r'(?<=[.!?])\s+', clean_text)
sentences = [s.strip() for s in sentences if len(s.strip()) > 0]

print("📌 Total sentences:", len(sentences))

# -------------------------
# CHUNKING WITH MIN 30 WORDS
# -------------------------
chunks = []
i = 0

while i < len(sentences):
    # Start with 2 sentences
    current_chunk = sentences[i:i+2]
    combined = " ".join(current_chunk)

    # Ensure >= 18 words (your original logic)
    while len(combined.split()) < 25 and (i + len(current_chunk)) < len(sentences):
        current_chunk.append(sentences[i + len(current_chunk)])
        combined = " ".join(current_chunk)

    chunks.append(combined)
    i += len(current_chunk)

print("📌 Total chunks created:", len(chunks))

# -------------------------
# SAVE TO CSV
# -------------------------
df = pd.DataFrame({"chunk": chunks})
df.to_csv(output_csv, index=False, encoding="utf-8")

print("✅ Chunk CSV saved at:", output_csv)


In [ ]:
# ================================================
# CONFIG
# ================================================
SPREADSHEET_ID = "1eNcH6DKc4N4pvS8Rmy3ZdsrOR6RcYzzwkYXop0Noi7A"

# ================================================
# LOAD GOOGLE SHEET DIRECTLY
# ================================================
csv_url = f"https://example.com/api"

df = pd.read_csv(csv_url)
script_chunks = df.iloc[:, 0].tolist()

print("Loaded", len(script_chunks), "script chunks")

In [ ]:
# ================================================
#  CONFIG
# ================================================

external_service_REDACTED_SECRET = 'REMOVED_FOR_PUBLIC_RELEASE'
LEONARDO_REDACTED_SECRET = 'REMOVED_FOR_PUBLIC_RELEASE'

client = external_service(api_key=external_service_API_KEY)

In [ ]:
VOICEOVER_FOLDER = "/content/drive/MyDrive/external_service/voice/"
FINAL_VIDEO = "/content/drive/MyDrive/external_service/Video/"

In [ ]:
import re
import random
from external_service import external_service

# ================================================
# 🔥 LOAD ENTIRE SCRIPT AS CONTEXT MEMORY
# ================================================
def build_script_memory(sentences):
    return " ".join(sentences)

# ================================================
# 🎨 RANDOM CINEMATIC VARIATION BOOSTER
# (added to enforce unique images)
# ================================================
def random_visual_style():
    styles = [
        "XXXXXXXXXXXXXXXXXX"
    ]
    return ", ".join(random.sample(styles, 3))


# ================================================
# ⚡ FIXED + UPGRADED: Generate mythology prompt
# ================================================
def create_prompt(text, full_script_memory):
    system_prompt = "USER_DEFINED_PROMPT_OMITTED""
You generate cinematic mythology-style image descriptions SAFE for Leonardo AI.



    # 🎨 NEW: Variation injected here
    variation = random_visual_style()

    user_prompt = f"""
Convert the SCRIPT CHUNK into a visually rich, mythology-style image prompt.

### VISUAL VARIATION TO APPLY FOR THIS IMAGE:
{variation}

FULL SCRIPT CONTEXT (reference only):
{full_script_memory}

SCRIPT CHUNK (actual content to convert):
{text}

Return only the final image prompt.
"""

    response = client.chat.completions.create(
        model="external_service-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.95,      # 🔥 Increased creativity
        max_tokens=250
    )

    return response.choices[0].message.content

In [ ]:
# ============================================================
# BUILD SCRIPT MEMORY (uses the same sentences from your chunking code)
# ============================================================
full_script_memory = build_script_memory(sentences)

# ============================================================
# GENERATE PROMPTS FOR EACH CHUNK
# ============================================================
prompts = []

for text in df.iloc[:, 0]:   # assuming script chunks are in column A
    if str(text).strip() == "":
        prompts.append("")
        continue

    # pass both required arguments
    prompt_out = create_prompt(str(text), full_script_memory)
    prompts.append(prompt_out)

df["prompt_generated"] = prompts

print("✅ Prompts generated for all rows!")
print(df.head())


In [ ]:
df.to_csv("/content/drive/MyDrive/external_service/script/script_chunks_with_prompts.csv", index=False)

In [ ]:
import os

image_folder = "/content/drive/MyDrive/external_service/image/"
os.makedirs(image_folder, exist_ok=True)   # creates folder if not exists

print("✅ Image folder ready:", image_folder)

In [ ]:
## Fish Audio TTS – Deep Story by Moses Nwosah, High Quality, Separate Chunks

import os
import pandas as pd
import requests

# -------------------------------
# PATHS
# -------------------------------
drive_root = "/content/drive/MyDrive/external_service"
voice_folder = os.path.join(drive_root, "voice")
os.makedirs(voice_folder, exist_ok=True)

# -------------------------------
# LOAD GOOGLE SHEET
# -------------------------------
file_id = "12oIWWen6aRBInJuyVrLDHlcmQ5gx7Mxt"
sheet_url = f"https://example.com/api"

df = pd.read_csv(sheet_url)

if "chunk" not in df.columns:
    raise ValueError("❌ 'chunk' column not found in Google Sheet")

print(f"📌 Loaded {len(df)} chunks from Google Sheet")

# -------------------------------
# FISH AUDIO SETTINGS
# -------------------------------
REDACTED_SECRET = 'REMOVED_FOR_PUBLIC_RELEASE'
REFERENCE_ID = "XXXXXXXXXXX" 
MODEL = "XXX"  

headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {API_KEY}"
}

url = "https://example.com/api"

# -------------------------------
# GENERATE VOICE (ONE FILE PER CHUNK)
# -------------------------------
for index, row in df.iterrows():

    text = str(row["chunk"]).strip()

    if len(text) < 2:
        print(f"⚠️ Skipping empty chunk at row {index+1}")
        continue

    payload = {
        "model": MODEL,
        "reference_id": REFERENCE_ID,
        "format": "mp3",
        "text": text,

        # Cinematic pacing
        "speed": 0.88,          # slower narration
        "temperature": 0.9,
        "top_p": 0.9,

        # Audio quality
        "audio_quality": "high",
        "silence": 0.28,        # slightly longer pauses
        "normalize": False
    }

    print(f"🎤 Generating voiceover for chunk {index + 1}…")

    response = requests.post(url, json=payload, headers=headers)

    if response.status_code == 200:
        output_path = os.path.join(voice_folder, f"chunk_{index+1}.mp3")
        with open(output_path, "wb") as f:
            f.write(response.content)
        print(f"✅ Saved chunk_{index+1}.mp3")
    else:
        print(f"❌ Error for chunk {index+1}: {response.text}")

print("⚔️ All Deep Story voices generated successfully — exactly like UI sample!")


In [ ]:
pip install moviepy pydrive

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:

# ============================================================
# FINAL CINEMATIC PIPELINE — OPTION A (BEST FOR external_service VOICE SYNC)
# Perfect synced voice → images match exactly → clean fades
# ============================================================

import os
import math
import random
import warnings
from tqdm import tqdm
from multiprocessing.pool import ThreadPool
import numpy as np
import cv2

from external_service import (
    ImageClip, VideoFileClip, AudioFileClip,
    CompositeVideoClip, CompositeAudioClip,
    concatenate_videoclips, concatenate_audioclips, VideoClip, AudioClip
)
from moviepy.video.fx import all as vfx
from moviepy.audio.fx.all import audio_fadein, audio_fadeout
from proglog import ProgressBarLogger
from natsort import natsorted

warnings.filterwarnings("ignore", category=UserWarning)

# --------------------------
# PATHS
# --------------------------
drive_root = "/content/drive/MyDrive/external_service"
image_folder = os.path.join(drive_root, "image")
voice_folder = os.path.join(drive_root, "voice")
bg_folder = os.path.join(drive_root, "bgvoice")
effect_folder = os.path.join(drive_root, "effect")
output_folder = os.path.join(drive_root, "Video")
os.makedirs(output_folder, exist_ok=True)
output_path = os.path.join(output_folder, "final_video.mp4")

TARGET_RES = (1920, 1080)
THREADS = 4

MOTION_BLUR_STRENGTH = 0.6
GRAIN_INTENSITY = 0.03
DEPTH_PARALLAX_DURATION = 180.0
DEPTH_PARALLAX_STRENGTH = 22
COLOR_GRADE_STRENGTH = 0.18
SUBTLE_CONTRAST = 1.03

FADE_DURATION = 0.6  # fade between images (NO overlap)


# --------------------------
# LOAD FILES
# --------------------------
def sorted_files(folder, exts):
    if not os.path.exists(folder):
        return []
    return natsorted([os.path.join(folder, f) for f in os.listdir(folder) if f.lower().endswith(exts)])

images = sorted_files(image_folder, (".png", ".jpg", ".jpeg"))
voice_files = sorted_files(voice_folder, (".mp3", ".wav", ".m4a", ".mp4"))
bg_files = sorted_files(bg_folder, (".mp3", ".wav"))

if not images: raise FileNotFoundError("No images found.")
if not voice_files: raise FileNotFoundError("No voice files found.")
if not bg_files: raise FileNotFoundError("No background music found.")

print(f"Found {len(images)} images, {len(voice_files)} voice chunks.")


# --------------------------
# LOAD OPTIONAL PARTICLES
# --------------------------
# particle_path = os.path.join(effect_folder, "particle.mp4")
# particle_base = None
# if os.path.exists(particle_path):
#     try:
#         particle_base = VideoFileClip(particle_path)
#         print("Loaded particle effect.")
#     except:
#         particle_base = None


# ================================================================
# STEP 1 — COMBINE ALL VOICE CHUNKS INTO ONE PERFECT VOICE TRACK
# ================================================================
print("\n🔊 Combining all voice chunks...")

raw_voices = [AudioFileClip(v) for v in voice_files]
voice_durations = [v.duration for v in raw_voices]

processed_voices = [
    v.fx(audio_fadein, FADE_DURATION)
     .fx(audio_fadeout, FADE_DURATION)
    for v in raw_voices
]

full_voice = concatenate_audioclips(processed_voices)
print("Voice track combined.")




# def particle_fx_for_duration(duration):
#     if particle_base is None:
#         return None
#     if particle_base.duration < duration:
#         loops = int(duration // particle_base.duration) + 1
#         return concatenate_videoclips([particle_base]*loops).subclip(0, duration).resize(TARGET_RES)
#     return particle_base.subclip(0, duration).resize(TARGET_RES)


# ================================================================
# STEP 2 — BUILD CLIPS WITHOUT AUDIO (SYNC FROM VOICE DURATIONS)
# ================================================================
depth_counter = 0

def build_clip_for_image(args):
    image_path, idx = args
    duration = voice_durations[idx]

    img = load_and_fill_image(image_path)
    use_depth = depth_counter < DEPTH_PARALLAX_DURATION

    if use_depth:
        clip = apply_depth_parallax(img, duration)
    else:
        clip = ImageClip(img).set_duration(duration)

    # Cinematic motions
    zoom_s = random.uniform(1.06,1.09)
    zoom_e = random.uniform(1.10,1.14)
    pan_x = random.uniform(-0.02,0.02)*TARGET_RES[0]
    pan_y = random.uniform(-0.02,0.02)*TARGET_RES[1]
    rot = random.uniform(-0.6,0.6)

    clip = (
        clip.resize(lambda t: zoom_s + (zoom_e-zoom_s)*(t/duration))
            .set_position(lambda t: (pan_x*(t/duration), pan_y*(t/duration)))
            .rotate(rot)
    )

    # Cinematic effects
    clip = clip.fl_image(lambda f: apply_motion_blur_to_frame(f))
    clip = clip.fl_image(lambda f: add_film_grain(color_grade_frame(f)))

    # Clean fades (no overlap)
    clip = clip.fx(vfx.fadein, FADE_DURATION)
    clip = clip.fx(vfx.fadeout, FADE_DURATION)

    return clip


print("\n🎨 Creating image clips...")
args = [(images[i], i) for i in range(len(images))]
pool = ThreadPool(THREADS)
clips = list(tqdm(pool.imap(build_clip_for_image, args), total=len(args)))
pool.close()
pool.join()


# ================================================================
# STEP 3 — CONCATENATE CLIPS (NO OVERLAP)
# ================================================================
print("\n🎬 Concatenating...")
video_clip = concatenate_videoclips(clips, method="compose")


# ================================================================
# STEP 4 — FINAL AUDIO (ONE PERFECT VOICE TRACK)
# ================================================================
print("\n🎵 Adding voice + background music...")

full_voice = full_voice.set_duration(video_clip.duration)

bg = AudioFileClip(bg_files[0]).volumex(0.1)
if bg.duration < video_clip.duration:
    loops = int(video_clip.duration // bg.duration) + 1
    bg = concatenate_audioclips([bg] * loops).subclip(0, video_clip.duration)
else:
    bg = bg.subclip(0, video_clip.duration)

final_audio = CompositeAudioClip([full_voice, bg]).set_duration(video_clip.duration)
video_clip = video_clip.set_audio(final_audio)


# ================================================================
# STEP 5 — EXPORT VIDEO
# ================================================================
print("\n🚀 Rendering final video...\n")

video_clip.write_videofile(
    output_path,
    fps=24,
    codec="libx264",
    audio_codec="aac",
    preset="ultrafast",
    threads=8,
    verbose=True
)

print("\n✅ DONE — Saved at:", output_path)